# OpenVTO Example 01: Avatar Generation

Generate a studio-quality avatar from a selfie and full-body posture image.

This is the first step in the virtual try-on pipeline.


## Setup

Install OpenVTO and configure your API key:


In [ ]:
# Install OpenVTO (uncomment if needed)
# !pip install openvto

import os
# Set your API key (or use mock provider for testing)
os.environ["GOOGLE_API_KEY"] = "your-api-key-here"


In [ ]:
from openvto import OpenVTO
from pathlib import Path
from IPython.display import Image, display


## Initialize Client

Create an OpenVTO client. Use `provider="mock"` for testing without API keys.


In [ ]:
# Use mock provider for testing (no API key needed)
# Change to provider="google" for real generation
vto = OpenVTO(provider="mock", cache_enabled=False)

print(f"Provider: {vto.provider_name}")
print(f"Image model: {vto.image_model}")
print(f"Video model: {vto.video_model}")


## Prepare Input Images

You need two images:
1. **Selfie** — A clear face photo for identity
2. **Posture** — A full-body image showing the desired pose


In [ ]:
# Helper to create test images for demo
def create_test_image(width=100, height=178, color=(100, 150, 200)):
    """Create a minimal test PNG."""
    import struct, zlib
    
    signature = b"\x89PNG\r\n\x1a\n"
    ihdr_data = struct.pack(">IIBBBBB", width, height, 8, 2, 0, 0, 0)
    ihdr_crc = zlib.crc32(b"IHDR" + ihdr_data) & 0xFFFFFFFF
    ihdr = struct.pack(">I", 13) + b"IHDR" + ihdr_data + struct.pack(">I", ihdr_crc)
    
    raw = b"".join(b"\x00" + bytes(color) * width for _ in range(height))
    compressed = zlib.compress(raw, 9)
    idat_crc = zlib.crc32(b"IDAT" + compressed) & 0xFFFFFFFF
    idat = struct.pack(">I", len(compressed)) + b"IDAT" + compressed + struct.pack(">I", idat_crc)
    
    iend_crc = zlib.crc32(b"IEND") & 0xFFFFFFFF
    iend = struct.pack(">I", 0) + b"IEND" + struct.pack(">I", iend_crc)
    
    return signature + ihdr + idat + iend

# Create test images (replace with your own paths)
selfie_image = create_test_image(color=(220, 180, 160))   # Skin tone
posture_image = create_test_image(color=(100, 120, 180)) # Blue tone

print("✓ Test images created")
print("  (Replace with real image paths for actual use)")


## Generate Avatar

Call `generate_avatar()` with your images. Options:
- `background`: "studio" (default), "white", "gradient"
- `seed`: For reproducible results
- `prompt`: Override the default prompt


In [1]:
# Generate the avatar
result = vto.generate_avatar(
    selfie=selfie_image,
    posture=posture_image,
    background="studio",
    seed=42,  # For reproducibility
)

print("✓ Avatar generated!")
print(f"  Size: {result.avatar.width}x{result.avatar.height}")
print(f"  Background: {result.avatar.background.value}")


NameError: name 'vto' is not defined

## View Results

Display the generated avatar:


In [ ]:
# Display the avatar image
display(Image(data=result.image, width=300))


## Generation Metadata

Examine details about the generation:


In [ ]:
meta = result.meta

print("Generation Metadata:")
print(f"  Model: {meta.model}")
print(f"  Provider: {meta.provider}")
print(f"  Seed: {meta.seed}")
print(f"  Latency: {meta.latency_ms:.0f}ms")
print(f"  Cache hit: {meta.cache_hit}")
if meta.cache_key:
    print(f"  Cache key: {meta.cache_key[:32]}...")


## Save Output

Save the generated avatar to disk:


In [ ]:
output_path = Path("avatar_output.png")
output_path.write_bytes(result.image)
print(f"✓ Saved to: {output_path.absolute()}")


## Next Steps

Now that you have an avatar, you can:

1. **Try-on clothing** — Apply clothing to your avatar
2. **Generate video** — Create animated video loops
3. **Run full pipeline** — End-to-end avatar → try-on → video

```python
# Quick preview of try-on
tryon = vto.generate_tryon(avatar=result, clothes=["shirt.jpg", "pants.jpg"])

# Or run the full pipeline
pipeline_result = vto.pipeline(
    selfie=selfie_image,
    posture=posture_image,
    clothes=["shirt.jpg"],
    make_video=True
)
```
